In [1]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from random import sample
from copy import deepcopy
import openbabel as ob
warnings.simplefilter("ignore")

In [2]:
slab = mb.load("ZnS.14sqnm.pdb")
# slab.save("lammps_test/ZnS.xyz")

In [3]:
slab

<Compound 850 particles, 1352 bonds, non-periodic, id: 140452689234432>

slab.visualize(backend="nglview", show_ports=True)

In [6]:
pmd_slab = slab.to_parmed()
pmd_slab.write_pdb("test_ZnS_slab.pdb")

# ob_conversion = ob.OBConversion()
# ob_conversion.SetInAndOutFormats("pdb", "bgf")
# molecule = ob.OBMol()

# ob_conversion.ReadFile(molecule, "test_ZnS_slab.pdb")
# ob_conversion.WriteFile(molecule, "test_ZnS_slab.bgf")

In [7]:
oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
oleic.add(mb.Port(anchor=oleic[18], separation=0.25, orientation=(oleic[18].pos - oleic[17].pos)))

In [8]:
pmd_oleic = oleic.to_parmed()
pmd_oleic.write_pdb("oleic_test.pdb")

# ob_conversion = ob.OBConversion()
# ob_conversion.SetInAndOutFormats("pdb", "bgf")
# molecule = ob.OBMol()

# ob_conversion.ReadFile(molecule, "oleic_test.pdb")
# ob_conversion.WriteFile(molecule, "oleic_test.bgf")

In [9]:
oleic.visualize(show_ports=True, backend="nglview")

NGLWidget()

## OK, works with only ligand and one slab

In [10]:
from random import sample
sites = sample(slab.all_ports(), k=14)

for site in sites:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.25, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    # oleic.remove_bond((site.anchor, oleic[18]))
    # slab.remove_bond((site.anchor, oleic[18]))
    slab.add(oleic, label="OLC[$]")

slab.visualize(show_ports=True, backend="nglview")

counter_slab.visualize()

In [11]:
system = mb.Compound()
system.add(slab)
# system.add(counter_slab)


In [12]:
system.visualize(backend="nglview")

NGLWidget()

In [17]:
system.save("test_slabs.xyz")

In [15]:
counter_slab = mb.load("ZnS.14sqnm.pdb")
for zn in counter_slab:
    if zn.element.symbol == 'Zn' and round(zn.pos[2],3) == 0.545:
        #print(zn)
        port = mb.Port(anchor=zn, orientation=[0,0,1], separation=0.05)
        counter_slab.add(port)

sites = sample(counter_slab.all_ports(), k=5)

for site in sites:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.25, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    # oleic.remove_bond((site.anchor, oleic[18]))
    # slab.remove_bond((site.anchor, oleic[18]))
    counter_slab.add(oleic, label="OLC[$]")

counter_slab.rotate(np.pi, [1,1,0])
counter_slab.translate([0,0,4])

In [16]:
pmd_counter = counter_slab.to_parmed()
pmd_counter.write_pdb("slab5ligs.pdb")

ob_conversion = ob.OBConversion()
ob_conversion.SetInAndOutFormats("pdb", "bgf")
molecule = ob.OBMol()

ob_conversion.ReadFile(molecule, "slab5ligs.pdb")
ob_conversion.WriteFile(molecule, "slab5ligs.bgf")

True

In [17]:
counter_slab.visualize(backend="nglview")

NGLWidget()

In [29]:
slab.visualize(backend='nglview')

NGLWidget()

## I think this is where my current code starts

In [17]:
test_slab = mb.load("ZnS.14sqnm.pdb")
for S in test_slab:
    if S.element.symbol == 'S' and round(S.pos[2],3) == 0.681:
        # print(S)
        port = mb.Port(anchor=S, orientation=[0,0,1], separation=0.05)
        test_slab.add(port)

In [18]:
test_slab.visualize(show_ports=True, backend="nglview")

NGLWidget()

In [19]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import random

def select_coordinates(coords, num_to_select, min_distance):
    num_coords = len(coords)
    if num_to_select > num_coords:
        print("Warning: Not enough coordinates available to select.")
        return None

    distance_matrix = squareform(pdist(coords))

    valid_indices = set(range(num_coords))
    selected_indices = []

    for _ in range(num_to_select):
        if not valid_indices:
            print("Warning: Unable to find enough coordinates with the given constraints.")
            break

        index = random.choice(list(valid_indices))
        selected_indices.append(index)
        valid_indices.remove(index)

        too_close_indices = set(np.where(distance_matrix[index] < min_distance)[0])
        valid_indices -= too_close_indices

    return selected_indices

In [20]:
ports = [port for port in test_slab.all_ports()]

In [21]:
coords = [port.pos for port in test_slab.all_ports()]

selected_sites = select_coordinates(coords, 28, 0.75)
ports = [test_slab.all_ports()[site] for site in selected_sites]

In [22]:
for site in ports:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.12, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    # oleic.remove_bond((site.anchor, oleic[18]))
    # slab.remove_bond((site.anchor, oleic[18]))
    test_slab.add(oleic, label="OLC[$]")

test_slab.rotate(np.pi, [1,1,0])
test_slab.translate([0,0,4])

In [23]:
test_slab.visualize(backend="nglview")

NGLWidget()

In [24]:
pmd_system = test_slab.to_parmed()
pmd_system.write_pdb("new_single_slab_28lig.pdb")

ob_conversion = ob.OBConversion()
ob_conversion.SetInAndOutFormats("pdb", "bgf")
molecule = ob.OBMol()

ob_conversion.ReadFile(molecule, "new_single_slab_28lig.pdb")
ob_conversion.WriteFile(molecule, "new_single_slab_28lig.bgf")

True

In [25]:
cool_slab = deepcopy(test_slab)
counter_slab = deepcopy(test_slab)
counter_slab.rotate(np.pi, [1,1,0])
counter_slab.translate([0,0,3])

system = mb.Compound()
system.add(cool_slab)
system.add(counter_slab)

In [26]:
system.visualize(backend="nglview")

NGLWidget()

In [27]:
pmd_system = system.to_parmed()
pmd_system.write_pdb("new_doubleslab_revised.pdb")

ob_conversion = ob.OBConversion()
ob_conversion.SetInAndOutFormats("pdb", "bgf")
molecule = ob.OBMol()

ob_conversion.ReadFile(molecule, "new_doubleslab_revised.pdb")
ob_conversion.WriteFile(molecule, "new_doubleslab_revised.bgf")

True

In [12]:
def assign_force_field_and_optimize(molecule, force_field_name):
    # Set up the force field
    force_field = ob.OBForceField.FindForceField(force_field_name)

    if force_field is None:
        print(f"Force field '{force_field_name}' not found.")
        return

    # Assign the force field atom types
    if not force_field.Setup(molecule):
        print("Error assigning force field atom types.")
        return

    # Optimize the geometry
    force_field.ConjugateGradients(1000)  # Number of optimization steps
    force_field.GetCoordinates(molecule)


In [ ]:
input_file = "test_slabs.xyz"
output_file = "minimized_slabs.xyz"

ob_conversion = ob.OBConversion()
ob_conversion.SetInAndOutFormats("xyz", "xyz")
molecule = ob.OBMol()

# Read the input file
if not ob_conversion.ReadFile(molecule, input_file):
    print(f"Error reading file '{input_file}'.")

In [ ]:
# Assign force field atom types and optimize the geometry
force_field_name = "uff"  # Use "mmff94" or "ghemical" for other force fields
assign_force_field_and_optimize(molecule, force_field_name)

# Save the optimized molecule
if not ob_conversion.WriteFile(molecule, output_file):
    print(f"Error writing file '{output_file}'.")

In [19]:
for comp in slab['OLC']:
    print(comp.pos)

[3.38371797 2.30064471 1.51393347]
[1.47611792 3.66324479 1.51393347]
[2.29361792 0.12054474 1.51393347]
[1.74861784 2.84564478 1.51393347]
[0.1135179  1.75564475 1.51393347]
[2.02111788 1.48314471 1.51393347]
[0.38601787 2.57314474 1.51393347]
[1.74861784 3.39074476 1.51393347]
[-0.15898211  0.39304475  1.51393347]
[0.65861793 1.21064479 1.51393347]
[2.83871789 3.39074476 1.51393347]
[-0.15898211  2.57314474  1.51393347]
[2.56611796 2.57314474 1.51393347]
[3.11121793 0.39304475 1.51393347]
[-0.43148212  3.39074476  1.51393347]
[2.83871789 0.12054474 1.51393347]
[0.93111785 2.57314474 1.51393347]
[0.65861793 0.66554476 1.51393347]
[2.83871789 1.75564475 1.51393347]
[-0.43148212  3.93574483  1.51393347]
[2.56611796 0.39304475 1.51393347]
[3.11121793 1.48314471 1.51393347]
[3.38371797 0.12054474 1.51393347]
[0.1135179  2.30064471 1.51393347]
[0.93111785 1.48314471 1.51393347]
[1.74861784 2.30064471 1.51393347]
[1.74861784 3.93574483 1.51393347]
[0.65861793 3.39074476 1.51393347]


In [16]:
system.save("test_slabs.mol2")

OSError: test_slabs.mol2 exists; not overwriting

In [13]:
pmd_system = system.to_parmed()

In [16]:
pmd_system.bonds

TrackedList([
	<Bond <Atom S [537]; In RES 0>--<Atom ZN [0]; In RES 0>; type=None>
	<Bond <Atom S [534]; In RES 0>--<Atom ZN [0]; In RES 0>; type=None>
	<Bond <Atom S [536]; In RES 0>--<Atom ZN [0]; In RES 0>; type=None>
	<Bond <Atom S [535]; In RES 0>--<Atom ZN [0]; In RES 0>; type=None>
	<Bond <Atom ZN [12]; In RES 0>--<Atom S [537]; In RES 0>; type=None>
	<Bond <Atom ZN [10]; In RES 0>--<Atom S [536]; In RES 0>; type=None>
	<Bond <Atom ZN [8]; In RES 0>--<Atom S [535]; In RES 0>; type=None>
	<Bond <Atom S [538]; In RES 0>--<Atom ZN [1]; In RES 0>; type=None>
	<Bond <Atom S [338]; In RES 0>--<Atom ZN [1]; In RES 0>; type=None>
	<Bond <Atom S [540]; In RES 0>--<Atom ZN [1]; In RES 0>; type=None>
	<Bond <Atom S [539]; In RES 0>--<Atom ZN [1]; In RES 0>; type=None>
	<Bond <Atom ZN [8]; In RES 0>--<Atom S [338]; In RES 0>; type=None>
	<Bond <Atom ZN [10]; In RES 0>--<Atom S [338]; In RES 0>; type=None>
	<Bond <Atom ZN [13]; In RES 0>--<Atom S [338]; In RES 0>; type=None>
	<Bond <Atom S [

In [14]:
pmd_system.write_pdb("single_slab_14lig.pdb")

ob_conversion = ob.OBConversion()
ob_conversion.SetInAndOutFormats("pdb", "bgf")
molecule = ob.OBMol()

ob_conversion.ReadFile(molecule, "single_slab_14lig.pdb")
ob_conversion.WriteFile(molecule, "single_slab_14lig.bgf")

True

## Trying to get periodicity to actually work

In [ ]:
mini_slab = mb.load("ZnS.mol2")

In [ ]:
mini_slab.visualize()

In [ ]:
mini_slab

In [ ]:
for i in mini_slab:
    print(i.pos)

In [80]:
unit_cell = mb.Box([0.545027, 0.545027, 0.545027], [90, 90, 90])

In [81]:
mini_slab.box = unit_cell

In [82]:
mini_slab

<Compound 46 particles, System box: Box: Lx=0.545027, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 56 bonds, id: 140311922202704>

In [83]:
mini_slab.periodicity=(True,True,True)

In [58]:
mini_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [84]:
mini_tiles = TiledCompound(mini_slab, (2,1,1))
mini_tiles.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [85]:
mini_tiles

<Compound2-1-1 92 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 112 bonds, id: 140311789801536>

In [65]:
mini_tiles[0].pos

array([0., 0., 0.])

In [75]:
pos1 = [round(p,2) for p in mini_tiles[0].pos]
pos1

[0.0, 0.0, 0.0]

In [90]:
matches = []
for i,part1 in enumerate(mini_tiles):
    pos1 = [round(p,1) for p in part1.pos]
    for j,part2 in enumerate(mini_tiles):
        pos2 = [round(p,1) for p in part2.pos]
        if j != i and (part1,part2) not in matches and (part2,part1) not in matches:
            if pos1 == pos2:
                print(i,j, part1.pos)
                matches.append([part1,part2))

4 46 [0.54503 0.      0.     ]
5 47 [0.54503 0.      0.54503]
6 48 [0.54503 0.54503 0.     ]
7 49 [0.54503 0.54503 0.54503]
9 54 [0.54503    0.27250999 0.27250999]
15 74 [0.40876999 0.40876999 0.40876999]
16 65 [0.40876999 0.13626    0.13626   ]
32 64 [ 0.40876999 -0.13626    -0.13626   ]
33 66 [ 0.68128002 -0.13626     0.13626   ]
34 67 [ 0.68128002  0.13626    -0.13626   ]
35 68 [ 0.40876999 -0.13626     0.40876999]
36 69 [0.40876999 0.13626    0.68128002]
37 70 [ 0.68128002 -0.13626     0.68128002]
38 60 [0.68128002 0.13626    0.40876999]
39 71 [ 0.40876999  0.40876999 -0.13626   ]
40 72 [0.40876999 0.68128002 0.13626   ]
41 63 [0.68128002 0.40876999 0.13626   ]
42 73 [ 0.68128002  0.68128002 -0.13626   ]
43 75 [0.40876999 0.68128002 0.68128002]
44 76 [0.68128002 0.40876999 0.68128002]
45 77 [0.68128002 0.68128002 0.40876999]


In [93]:
for match in matches:
    mini_tiles.remove(match[1])

In [94]:
mini_tiles

<Compound2-1-1 71 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 84 bonds, id: 140311789801536>

In [95]:
mini_tiles.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
for port in mini_tiles.all_ports()

In [96]:
mini_tiles.all_ports()

[<Port, anchor: 'ZN', labels: , id: 140311788749776>,
 <Port, anchor: 'ZN', labels: , id: 140311788749680>,
 <Port, anchor: 'ZN', labels: , id: 140311786503616>,
 <Port, anchor: 'ZN', labels: , id: 140311788751408>,
 <Port, anchor: 'ZN', labels: , id: 140311786555616>,
 <Port, anchor: 'ZN', labels: , id: 140311788749392>,
 <Port, anchor: 'ZN', labels: , id: 140311788814448>,
 <Port, anchor: 'ZN', labels: , id: 140311788751888>]

In [78]:
mini_tiles.save("min_test.xyz")

In [79]:
mini_tiles.energy_minimize()

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested

In [2]:
slab = 
for zn in slab:
    if zn.element.symbol == 'Zn' and round(zn.pos[2],3) == 0.545:
        print(zn)
        port = mb.Port(anchor=zn, orientation=[0,0,1], separation=0.05)
        slab.add(port)

/home/rramji/.conda/envs/openmm/lib/python3.9/site-packages/mbuild/conversion.py:481: UserWarning: Using parmed reader. Bonds may be inferred from inter-particle distances and standard residue templates. Please check that the bonds in mb.Compound are accurate
  warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
import mbuild as mb
from mbuild.lib.surfaces import Betacristobalite

surface = Betacristobalite()
tiled_surface = mb.lib.recipes.TiledCompound(surface, n_tiles=(2, 1, 1))


#######################################################
The code at conversion.py:429 requires the gmso package
#######################################################


DelayImportError: The code at conversion.py:429 requires the gmso package